In [63]:
import pandas as pd
import numpy as np
import pickle


In [64]:
df=pd.read_csv('/home/rahulpuppala/Documents/Major Project 2/artifacts/cleaned_data.csv')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [65]:
with open("/home/rahulpuppala/Documents/Major Project 2/artifacts/preprocessor.pkl",'rb') as file:
    preprocessor=pickle.load(file)

In [66]:
train_df=pd.read_csv("/home/rahulpuppala/Documents/Major Project 2/artifacts/train.csv")
test_df=pd.read_csv("/home/rahulpuppala/Documents/Major Project 2/artifacts/test.csv")


In [67]:
train_df.drop(columns=['Unnamed: 0'],inplace=True)
test_df.drop(columns=['Unnamed: 0'],inplace=True)

In [68]:
from tensorflow.keras.utils import to_categorical
target_feature="Churn"

input_feature_train_df=train_df.drop(columns=[target_feature],axis=1)
y_train=train_df[target_feature].replace({'Yes':1,'No':0}).astype(int)

input_feature_test_df=test_df.drop(columns=[target_feature],axis=1)
y_test=test_df[target_feature].replace({'Yes':1,'No':0}).astype(int)


# y_train = to_categorical(y_train, num_classes=17)
# y_test = to_categorical(y_test, num_classes=17)

X_train=preprocessor.fit_transform(input_feature_train_df)
X_test=preprocessor.transform(input_feature_test_df)


/tmp/ipykernel_5540/2720620802.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train=train_df[target_feature].replace({'Yes':1,'No':0}).astype(int)
/tmp/ipykernel_5540/2720620802.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test=test_df[target_feature].replace({'Yes':1,'No':0}).astype(int)


In [69]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [70]:
model=Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid') # output layer
]
)

In [71]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,993 (19.50 KB)

 Trainable params: 4,993 (19.50 KB)

 Non-trainable params: 0 (0.00 B)

In [72]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()

In [73]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [74]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [75]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [76]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7629 - loss: 0.4761 - val_accuracy: 0.8048 - val_loss: 0.4295
Epoch 2/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7994 - loss: 0.4293 - val_accuracy: 0.8027 - val_loss: 0.4322
Epoch 3/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7876 - loss: 0.4409 - val_accuracy: 0.7984 - val_loss: 0.4173
Epoch 4/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8048 - loss: 0.4184 - val_accuracy: 0.7921 - val_loss: 0.4280
Epoch 5/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8005 - loss: 0.4199 - val_accuracy: 0.8041 - val_loss: 0.4286
Epoch 6/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8077 - loss: 0.4175 - val_accuracy: 0.7999 - val_loss: 0.4265
Epoch 7/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8074 - loss: 0.4136 - val_accuracy: 0.8041 - val_loss: 0.4179
Epoch 8/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8167 - loss: 0.4005 - val_accu

From the above observation, we can say that the accuracy is 80% using ann

In [77]:
# model prediction
data={
    'gender':'male',
    'SeniorCitizen':0,
    'Partner':'No',
    'Dependents':'No',
    'tenure':'9',
    'PhoneService':'No',
    'MultipleLines': 'No',
    'InternetService':'No',
    'OnlineSecurity':'No',
    'OnlineBackup':'Yes',	
    'DeviceProtection':'No',
    'TechSupport':'No',
    'StreamingTV':'Yes',
    'StreamingMovies':'Yes',
    'Contract':'One Year',
    'PaperlessBilling':'Yes',
    'PaymentMethod': 'Electronic check',
    'MonthlyCharges':111.3,
    'TotalCharges': 5450.7

}

In [78]:
new_data=pd.DataFrame(data,index=[0])

In [79]:
new_data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,male,0,No,No,9,No,No,No,No,Yes,No,No,Yes,Yes,One Year,Yes,Electronic check,111.3,5450.7


In [80]:
new_data=preprocessor.transform(new_data)

In [81]:
new_data

array([[-0.94328209,  1.55056523,  1.40408367,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ]])

In [82]:
predict_churn=model.predict(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


In [ ]:
predict_churn

array([[0.49835366]], dtype=float32)

In [85]:
if predict_churn > 0.5:
    print("the above customer is likely to exit your services.")
else:
    print("The above customer is going to continue his services")

The above customer is going to continue his services


In [84]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,981 (58.52 KB)

 Trainable params: 4,993 (19.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,988 (39.02 KB)